<a href="https://colab.research.google.com/github/Akshatasthana18/Celebal25/blob/main/loan_Approval_ai_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sonalisingh1411/loan-approval-prediction")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/loan-approval-prediction


In [33]:
!pip install pandas faiss-cpu sentence-transformers openai transformers


In [34]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import openai
from transformers import pipeline
import os

# Load dataset
# The path from kagglehub.dataset_download is a directory, not a file.
# We need to find the actual CSV file within this directory.
dataset_dir = "/root/.cache/kagglehub/datasets/sonalisingh1411/loan-approval-prediction/versions/1"
csv_file_path = os.path.join(dataset_dir, "Training Dataset.csv") # Using the correct CSV file name

df = pd.read_csv(csv_file_path)
df.fillna("NA", inplace=True)

# Convert rows into readable text for retrieval
def row_to_text(row):
    return " | ".join(f"{col}: {row[col]}" for col in df.columns)

documents = [row_to_text(row) for _, row in df.iterrows()]

# Step 1: Embed Documents
embed_model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight and fast
doc_embeddings = embed_model.encode(documents, convert_to_numpy=True)

# Step 2: Create FAISS Index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# Step 3: Retrieval Function
def retrieve_docs(query, top_k=5):
    query_vec = embed_model.encode([query])
    distances, indices = index.search(query_vec, top_k)
    return [documents[i] for i in indices[0]]

# Step 4: Generate Response
def generate_answer(query, context):
    try:
        # Replace with your OpenAI API key
        openai.api_key = "your-openai-key"
        response = openai.ChatCompletion.create(
            model="gpt-4",  # or gpt-3.5-turbo
            messages=[
                {"role": "system", "content": "You are a helpful assistant for loan-related questions."},
                {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}"}
            ],
            temperature=0.5,
            max_tokens=300,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print("OpenAI failed:", e)
        # fallback using Hugging Face if OpenAI fails
        fallback_pipe = pipeline("text-generation", model="google/flan-t5-small")
        return fallback_pipe(f"Context: {context}\n\nQuestion: {query}", max_length=300)[0]['generated_text']

# Step 5: Ask a Question
def ask_bot(question):
    context_docs = retrieve_docs(question)
    context = "\n".join(context_docs)
    answer = generate_answer(question, context)
    return answer

# Example
if __name__ == "__main__":
    while True:
        user_q = input("\nAsk your loan-related question (or type 'exit'): ")
        if user_q.lower() == "exit":
            break
        print("\nBot:", ask_bot(user_q))

/tmp/ipython-input-34-3928595618.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NA' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("NA", inplace=True)



Ask your loan-related question (or type 'exit'): exit
